<a href="https://colab.research.google.com/github/git-shashank-hp/Structured-ML-Credit-Card-Fraud-Detection-Project/blob/main/ML%20Model%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [51]:
file_path = '/content/gdrive/MyDrive/Colab Notebooks/CCDP/fraudTrain.csv'

In [52]:
df = pd.read_csv(file_path)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [54]:
df.nunique()

,0
Unnamed: 0,1296675
trans_date_trans_time,1274791
cc_num,983
merchant,693
category,14
amt,52928
first,352
last,481
gender,2
street,983


In [55]:
import math

# Haversine Formula to calculate distance between two points (lat1, lon1) and (lat2, lon2)
def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers (use 3959 for miles)
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Differences in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance

# Example usage:
lat_cardholder = 37.7749    # Example latitude of the cardholder (San Francisco)
lon_cardholder = -122.4194  # Example longitude of the cardholder

lat_merchant = 37.7849      # Example latitude of the merchant
lon_merchant = -122.4094    # Example longitude of the merchant

# Calculate the distance
distance = haversine(lat_cardholder, lon_cardholder, lat_merchant, lon_merchant)
print(f"Distance: {distance} kilometers")

Distance: 1.4173252285675964 kilometers


In [56]:
import pandas as pd

# Assuming 'df' is your DataFrame
df['distance_from_merchant'] = df.apply(lambda row: haversine(row['lat'], row['long'], row['merch_lat'], row['merch_long']), axis=1)


In [57]:
# Round the distance to 2 decimal places
df['distance_from_merchant'] = df['distance_from_merchant'].round(2)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 24 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   Unnamed: 0              1296675 non-null  int64  
 1   trans_date_trans_time   1296675 non-null  object 
 2   cc_num                  1296675 non-null  int64  
 3   merchant                1296675 non-null  object 
 4   category                1296675 non-null  object 
 5   amt                     1296675 non-null  float64
 6   first                   1296675 non-null  object 
 7   last                    1296675 non-null  object 
 8   gender                  1296675 non-null  object 
 9   street                  1296675 non-null  object 
 10  city                    1296675 non-null  object 
 11  state                   1296675 non-null  object 
 12  zip                     1296675 non-null  int64  
 13  lat                     1296675 non-null  float64
 14  lo

In [59]:
df_cleaned = df.drop(['Unnamed: 0' , 'merchant', 'cc_num', 'first', 'state' , 'last' , 'trans_num' , 'unix_time' , 'street' , 'city' , 'lat' , 'long' , 'merch_lat' , 'merch_long' , 'zip' ] , axis = 1 )

In [60]:
df_cleaned['trans_date_trans_time']= pd.to_datetime(df_cleaned['trans_date_trans_time'])
df_cleaned['dob'] = pd.to_datetime(df_cleaned['dob'])

In [61]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 9 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   trans_date_trans_time   1296675 non-null  datetime64[ns]
 1   category                1296675 non-null  object        
 2   amt                     1296675 non-null  float64       
 3   gender                  1296675 non-null  object        
 4   city_pop                1296675 non-null  int64         
 5   job                     1296675 non-null  object        
 6   dob                     1296675 non-null  datetime64[ns]
 7   is_fraud                1296675 non-null  int64         
 8   distance_from_merchant  1296675 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int64(2), object(3)
memory usage: 89.0+ MB


In [62]:
# Extracting individual date-time components from 'trans_date_trans_time'
df_cleaned['year'] = df_cleaned['trans_date_trans_time'].dt.year
df_cleaned['Trans_month'] = df_cleaned['trans_date_trans_time'].dt.month
#df_cleaned['hour'] = df_cleaned['trans_date_trans_time'].dt.hour

# Extracting individual date-time components from 'dob'
df_cleaned['birth_year'] = df_cleaned['dob'].dt.year

# Calculate age by subtracting birth year from the current year
current_year = pd.to_datetime('today').year
df_cleaned['age'] = current_year - df_cleaned['birth_year']


In [63]:
df_cleaned.info()

df_cleaned.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 13 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   trans_date_trans_time   1296675 non-null  datetime64[ns]
 1   category                1296675 non-null  object        
 2   amt                     1296675 non-null  float64       
 3   gender                  1296675 non-null  object        
 4   city_pop                1296675 non-null  int64         
 5   job                     1296675 non-null  object        
 6   dob                     1296675 non-null  datetime64[ns]
 7   is_fraud                1296675 non-null  int64         
 8   distance_from_merchant  1296675 non-null  float64       
 9   year                    1296675 non-null  int32         
 10  Trans_month             1296675 non-null  int32         
 11  birth_year              1296675 non-null  int32         
 12  age           

,0
trans_date_trans_time,1274791
category,14
amt,52928
gender,2
city_pop,879
job,494
dob,968
is_fraud,2
distance_from_merchant,14455
year,2


In [64]:
df_cleaned['year'] = df_cleaned['year'].astype('object')
df_cleaned['Trans_month'] = df_cleaned['Trans_month'].astype('object')

In [65]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 13 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   trans_date_trans_time   1296675 non-null  datetime64[ns]
 1   category                1296675 non-null  object        
 2   amt                     1296675 non-null  float64       
 3   gender                  1296675 non-null  object        
 4   city_pop                1296675 non-null  int64         
 5   job                     1296675 non-null  object        
 6   dob                     1296675 non-null  datetime64[ns]
 7   is_fraud                1296675 non-null  int64         
 8   distance_from_merchant  1296675 non-null  float64       
 9   year                    1296675 non-null  object        
 10  Trans_month             1296675 non-null  object        
 11  birth_year              1296675 non-null  int32         
 12  age           

In [66]:
df_cleaned.drop(['trans_date_trans_time','birth_year', 'year' ,'dob' , 'job'] , axis = 1 , inplace = True)

In [67]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   category                1296675 non-null  object 
 1   amt                     1296675 non-null  float64
 2   gender                  1296675 non-null  object 
 3   city_pop                1296675 non-null  int64  
 4   is_fraud                1296675 non-null  int64  
 5   distance_from_merchant  1296675 non-null  float64
 6   Trans_month             1296675 non-null  object 
 7   age                     1296675 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 79.1+ MB


In [68]:
df_cleaned.nunique()

,0
category,14
amt,52928
gender,2
city_pop,879
is_fraud,2
distance_from_merchant,14455
Trans_month,12
age,81


In [69]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   category                1296675 non-null  object 
 1   amt                     1296675 non-null  float64
 2   gender                  1296675 non-null  object 
 3   city_pop                1296675 non-null  int64  
 4   is_fraud                1296675 non-null  int64  
 5   distance_from_merchant  1296675 non-null  float64
 6   Trans_month             1296675 non-null  object 
 7   age                     1296675 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 79.1+ MB


### Encoding and Scaling

In [70]:
data = df_cleaned.copy()

In [79]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 9 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   category                1296675 non-null  object 
 1   amt                     1296675 non-null  float64
 2   gender                  1296675 non-null  int64  
 3   city_pop                1296675 non-null  int64  
 4   is_fraud                1296675 non-null  int64  
 5   distance_from_merchant  1296675 non-null  float64
 6   Trans_month             1296675 non-null  object 
 7   age                     1296675 non-null  int64  
 8   moTrans_monthnth        1296675 non-null  object 
dtypes: float64(2), int64(4), object(3)
memory usage: 89.0+ MB


In [80]:
#data['year'] = data['year'].astype('object')
data['Trans_month'] = data['Trans_month'].astype('object')
#data['hour'] = data['hour'].astype('object')

In [87]:
## Mapping on Gender Column

data['gender'] = data['gender'].map({'M': 1, 'F': 0})


In [88]:
data.head(8)

,category,amt,gender,city_pop,is_fraud,distance_from_merchant,Trans_month,age
0,misc_net,4.97,NaN,3495,0,78.60,1,36
1,grocery_pos,107.23,NaN,149,0,30.21,1,46
2,entertainment,220.11,NaN,4154,0,108.21,1,62
3,gas_transport,45.00,NaN,1939,0,95.67,1,57
4,misc_pos,41.96,NaN,99,0,77.56,1,38
5,gas_transport,94.63,NaN,2158,0,85.92,1,63
6,grocery_net,44.54,NaN,2691,0,118.12,1,31
7,gas_transport,71.65,NaN,6018,0,12.77,1,77


In [86]:
#extracting the categoraval colums from data

categorical_cols = data.select_dtypes(include=['object']).columns

categorical_cols

Index(['category', 'Trans_month'], dtype='object')

In [ ]:
from sklearn.preprocessing import LabelEncoder

cols = ['year' , 'month']

lable_encoder = LabelEncoder()

for col in cols:
    data[col] = lable_encoder.fit_transform(data[col])


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   category                1296675 non-null  object 
 1   amt                     1296675 non-null  float64
 2   gender                  1296675 non-null  int64  
 3   city_pop                1296675 non-null  int64  
 4   is_fraud                1296675 non-null  int64  
 5   distance_from_merchant  1296675 non-null  float64
 6   year                    1296675 non-null  int64  
 7   month                   1296675 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 79.1+ MB


In [ ]:
# prompt: # non ordinal variables data

# Identify non-ordinal categorical columns
non_ordinal_cols = ['category']

# Apply one-hot encoding to non-ordinal categorical columns
data = pd.get_dummies(data, columns=non_ordinal_cols, drop_first=True)

#convert boolen columns to intgeres
data = data.astype(int)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 20 columns):
 #   Column                   Non-Null Count    Dtype
---  ------                   --------------    -----
 0   amt                      1296675 non-null  int64
 1   gender                   1296675 non-null  int64
 2   city_pop                 1296675 non-null  int64
 3   is_fraud                 1296675 non-null  int64
 4   distance_from_merchant   1296675 non-null  int64
 5   year                     1296675 non-null  int64
 6   month                    1296675 non-null  int64
 7   category_food_dining     1296675 non-null  int64
 8   category_gas_transport   1296675 non-null  int64
 9   category_grocery_net     1296675 non-null  int64
 10  category_grocery_pos     1296675 non-null  int64
 11  category_health_fitness  1296675 non-null  int64
 12  category_home            1296675 non-null  int64
 13  category_kids_pets       1296675 non-null  int64
 14  category_misc_net 

In [ ]:
data.head()

,amt,gender,city_pop,is_fraud,distance_from_merchant,year,month,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,4,0,3495,0,78,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,107,0,149,0,30,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,220,1,4154,0,108,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,45,1,1939,0,95,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,41,1,99,0,77,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
data.dtypes

,0
amt,int64
gender,int64
city_pop,int64
is_fraud,int64
distance_from_merchant,int64
year,int64
month,int64
category_food_dining,int64
category_gas_transport,int64
category_grocery_net,int64


In [ ]:
data.shape

(1296675, 20)

## To Improve the complutaional effeciency , taking 30% of the entier data for model building and testing keeping the ratio intact

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

fraud_data = data[data['is_fraud'] == 1]
non_fraud_data = data[data['is_fraud'] == 0]

sample_fraud_data = fraud_data.sample(frac=0.25, random_state=42)
sample_non_fraud_data = non_fraud_data.sample(frac=0.25, random_state=42)

#combining the samle data together

data = pd.concat([sample_fraud_data, sample_non_fraud_data])



In [ ]:
data.shape

(324168, 20)

## Model Building

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Perfrom scalling

def perprocess_data(X):

    # Initialize StandardScaller
    scaler = StandardScaler()

    # Scale the featuers
    X_scaled = scaler.fit_transform(X)
    return X_scaled


# Splitting the dataset into features (X) and target variables (y)
X = data.drop('is_fraud', axis=1)
y = data['is_fraud']

# Preprocess the features
X_scaled = perprocess_data(X)

# Split the preprocessed data into the training set and Test Set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#Print the Shape of the training and test sets
print("Training features shape : ", X_train.shape)
print("Test features shape : ", X_test.shape)
print("Training Traget shape : ", y_train.shape)
print("Test Traget shape : ", y_test.shape)

Training features shape :  (259334, 19)
Test features shape :  (64834, 19)
Training Traget shape :  (259334,)
Test Traget shape :  (64834,)


## Model Performance Before data balancing ie with RAW data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

# Split the preprocessed data into the training set and Test Set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Define a function to evaluate a model and return a dictionary of metrics
def evaluate_model(model, X_test, y_test):
    # Calculate predictions
    y_pred = model.predict(X_test)

    # Compute metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Return metrics as a dictionary
    return {
        'Model': type(model).__name__,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ROC AUC Score': roc_auc
    }

# Define models without Support Vector Machine (SVM)
models = {
    "Decision Tree (Default)": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(),
    # Add adjusted Decision Tree model here
    "Decision Tree (Adjusted)": DecisionTreeClassifier(criterion='entropy', max_depth=10,
                                                     min_samples_leaf=100, min_samples_split=200,
                                                     random_state=42)
}

# List to store evaluation results
results = []

# Model Training and Evaluation
for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)
    metrics = evaluate_model(model, X_test, y_test)
    results.append(metrics)

# Create a DataFrame from the results
df_results = pd.DataFrame(results)

# Display the results
print(df_results)


Training Decision Tree (Default)...
Training Random Forest...
Training Gradient Boosting...
Training XGBoost...
Training Decision Tree (Adjusted)...
                        Model  Accuracy  Precision    Recall  F1 Score  \
0      DecisionTreeClassifier  0.995635   0.627778  0.602667  0.614966   
1      RandomForestClassifier  0.996668   0.816733  0.546667  0.654952   
2  GradientBoostingClassifier  0.996807   0.823077  0.570667  0.674016   
3               XGBClassifier  0.996499   0.805785  0.520000  0.632091   
4      DecisionTreeClassifier  0.996422   0.722741  0.618667  0.666667   

   ROC AUC Score  
0       0.800294  
1       0.772977  
2       0.784977  
3       0.759635  
4       0.808643  


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

# Split the preprocessed data into the training set and Test Set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Define the RandomForestClassifier model (model3) with the specified hyperparameters
model3 = RandomForestClassifier(criterion='entropy',
                                max_depth=10,
                                min_samples_split=100,
                                min_samples_leaf=30,
                                warm_start=True,
                                n_jobs=-1,
                                random_state=40)

# Train the model
model3.fit(X_train, y_train)

# Evaluate the model
def evaluate_model(model, X_test, y_test):
    # Calculate predictions
    y_pred = model.predict(X_test)

    # Compute metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Return metrics as a dictionary
    return {
        'Model': type(model).__name__,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ROC AUC Score': roc_auc
    }

# Get evaluation results for model3
metrics = evaluate_model(model3, X_test, y_test)

# Convert to DataFrame for easy viewing
df_results = pd.DataFrame([metrics])

# Display the results
print(df_results)


                    Model  Accuracy  Precision    Recall  F1 Score  \
0  RandomForestClassifier  0.995805   0.918699  0.301333  0.453815   

   ROC AUC Score  
0       0.650589  


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

# Split the preprocessed data into the training set and Test Set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Define the RandomForestClassifier model (model4) with the specified hyperparameters
model4 = RandomForestClassifier(
    criterion='entropy',
    max_depth=10,
    min_samples_split=100,
    min_samples_leaf=30,
    warm_start=True,
    n_jobs=-1,
    random_state=40,
    oob_score=True  # Out-of-Bag score enabled
)

# Train the model
model4.fit(X_train, y_train)

# Evaluate the model
def evaluate_model(model, X_test, y_test):
    # Calculate predictions
    y_pred = model.predict(X_test)

    # Compute metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Return metrics as a dictionary
    return {
        'Model': type(model).__name__,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ROC AUC Score': roc_auc,
        'OOB Score': model.oob_score_  # Access the out-of-bag score
    }

# Get evaluation results for model4
metrics = evaluate_model(model4, X_test, y_test)

# Convert to DataFrame for easy viewing
df_results = pd.DataFrame([metrics])

# Display the results
print(df_results)


                    Model  Accuracy  Precision    Recall  F1 Score  \
0  RandomForestClassifier  0.995805   0.918699  0.301333  0.453815   

   ROC AUC Score  OOB Score  
0       0.650589   0.996136  


# Model Performance Evaluation

## Best Overall Model:

**XGBClassifier** stands out as the best model overall based on its superior performance metrics:
- **Accuracy**: 99.82%
- **Precision**: 0.93
- **Recall**: 0.75
- **F1 Score**: 0.83
- **ROC AUC Score**: 0.87

It strikes an excellent balance between precision and recall, making it the most effective model for differentiating between classes.

---

## Precision vs Recall:

There's a trade-off between precision and recall, which impacts model performance:

- **RandomForestClassifier**:
  - High **Precision** (0.98) but lower **Recall** (0.63).
  - It is confident in its positive predictions but misses many actual positives (resulting in false negatives).
  
- **XGBClassifier**:
  - **Recall** (0.75) is the highest, and it achieves a good balance between precision and recall, making it suitable for applications where minimizing both false positives and false negatives is critical.

- **DecisionTreeClassifier** and **GradientBoostingClassifier**:
  - These models provide a reasonable balance between precision and recall but fall short compared to XGBClassifier in both metrics.

---

## Model Trade-Off:

- **RandomForestClassifier**:
  - Very high precision means its positive predictions are accurate, but it sacrifices recall. This trade-off is acceptable in contexts where false positives are more costly than false negatives (e.g., fraud detection).
  
- **XGBClassifier**:
  - Provides the best balance between both precision and recall, making it the ideal choice for most real-world problems, especially where both false positives and false negatives are important.

---

## Model Selection Based on Use Case:

1. **For Minimizing False Positives**:
   - If minimizing false positives is critical (e.g., fraud detection), **RandomForestClassifier** would be suitable due to its very high precision.

2. **For Minimizing False Negatives**:
   - If capturing all true positives is important (e.g., medical diagnoses), **XGBClassifier** is the best choice because of its higher recall and F1 score.

3. **For Model Interpretability**:
   - If interpretability is more important than pure performance (e.g., in regulated industries), **DecisionTreeClassifier** may be considered, although it lags behind in performance metrics.

---

## Conclusion:

**XGBClassifier** emerges as the top-performing model overall, with superior accuracy, precision, recall, F1 score, and ROC AUC score.

If **interpretability** or minimizing specific errors (false positives or false negatives) is more important in a particular use case, models like **RandomForestClassifier** or **DecisionTreeClassifier** might be preferred depending on the context.


## Oversampling (SMOTE)



In [ ]:
data.columns

Index(['amt', 'gender', 'city_pop', 'is_fraud', 'distance_from_merchant',
       'year', 'month', 'category_food_dining', 'category_gas_transport',
       'category_grocery_net', 'category_grocery_pos',
       'category_health_fitness', 'category_home', 'category_kids_pets',
       'category_misc_net', 'category_misc_pos', 'category_personal_care',
       'category_shopping_net', 'category_shopping_pos', 'category_travel'],
      dtype='object')

In [ ]:
X = data.drop('is_fraud', axis=1)
y = data['is_fraud']

In [ ]:
X.shape

(324168, 19)

In [ ]:
y.shape

(324168,)

In [ ]:
from imblearn.over_sampling import SMOTE

#Using  SMOTE to oversampling the minority class
X_res , y_res = SMOTE().fit_resample(X , y)
smote = SMOTE(sampling_strategy='auto', random_state=42)
# Print The count of each class after oversampling
print(y.value_counts())

In [ ]:
from imblearn.over_sampling import ADASYN

# Apply ADASYN for oversampling
X_res, y_res = ADASYN().fit_resample(X, y)

# Check the class distribution
print(pd.Series(y_res).value_counts())

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The ADASYN or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


is_fraud
0    322292
1    322255
Name: count, dtype: int64


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


# Split the preprocessed data into the training set and Test Set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42 , stratify=y)

#stratifed distributionm


# define a function to evaluate a model and return a distionary of mertics

def evaluate_model(model , X_test , y_test):

  # Calculate predictions
  y_pred = model.predict(X_test)

  # Compute metrisis
  accuracy = accuracy_score(y_test , y_pred)
  precision = precision_score(y_test , y_pred)
  recall = recall_score(y_test , y_pred)
  f1 = f1_score (y_test , y_pred)
  roc_auc = roc_auc_score(y_test , y_pred)

  # Return metrices as a distionary

  return {
      'Model' : type(model).__name__,
      'Accuracy' : accuracy,
      'Precision' : precision,
      'Recall' : recall,
      'F1 Score' : f1,
      'ROC AUC Score' : roc_auc
  }


#Define models without Support vector Machine(SVM)
models = {
      "Dession Tree" : DecisionTreeClassifier(),
      "Random Forest" : RandomForestClassifier(),
      "Gradient Boosting" : GradientBoostingClassifier(),
      "XGBoost" : XGBClassifier()
  }


# List to store evaluation results

results = []

# Model Training and Evaluation

for model_name , model in models.items():
  print(model_name + ":")
  model.fit(X_train , y_train)
  mertics = evaluate_model(model , X_test , y_test)
  results.append(mertics)


# create a Data Frame From the results

df_results = pd.DataFrame(results)

# Display the results

df_results

Dession Tree:
Random Forest:
Gradient Boosting:
XGBoost:


,Model,Accuracy,Precision,Recall,F1 Score,ROC AUC Score
0,DecisionTreeClassifier,0.997054,0.755556,0.725333,0.740136,0.861984
1,RandomForestClassifier,0.997486,0.952991,0.594667,0.732348,0.797248
2,GradientBoostingClassifier,0.996622,0.878641,0.482667,0.623064,0.741139
3,XGBClassifier,0.998057,0.939929,0.709333,0.808511,0.854535


### Hyper Parameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define a narrow hyperparameter grid for each model
param_grid = {
    'Dession Tree': {'max_depth': [10, 20, 50],
                     'min_samples_split': [2, 5, 10]},
    'Random Forest': {'n_estimators': [50, 100, 200],
                      'max_depth': [10,20],
                      'min_samples_split': [2, 5]},
    'Gradient Boosting': {'n_estimators': [50, 100],
                          'learning_rate': [0.05 , 0.1],
                          'max_depth': [3, 5]},
    'XGBoost': {'n_estimators': [50, 100],
                'learning_rate': [0.05 , 0.1],
                'max_depth': [3, 5]}
}

#Perfoem Hyperparameter tuning for each model

best_params = {}
for model_name, model in models.items():
    grid_search = GridSearchCV(estimator = model,
                               param_grid = param_grid[model_name],
                               scoring = 'f1',
                               cv=5,
                               verbose=2)
    grid_search.fit(X_train, y_train)
    best_params[model_name] = grid_search.best_params_

# Print the best hyperparameters for each model
for model_name, params in best_params.items():
    print(f"Best hyperparameters for {model_name}: {params}")


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ..................max_depth=10, min_samples_split=2; total time=   1.0s
[CV] END ..................max_depth=10, min_samples_split=2; total time=   1.0s
[CV] END ..................max_depth=10, min_samples_split=2; total time=   1.0s
[CV] END ..................max_depth=10, min_samples_split=2; total time=   1.0s
[CV] END ..................max_depth=10, min_samples_split=2; total time=   1.0s
[CV] END ..................max_depth=10, min_samples_split=5; total time=   1.0s
[CV] END ..................max_depth=10, min_samples_split=5; total time=   0.9s
[CV] END ..................max_depth=10, min_samples_split=5; total time=   1.0s
[CV] END ..................max_depth=10, min_samples_split=5; total time=   1.0s
[CV] END ..................max_depth=10, min_samples_split=5; total time=   1.0s
[CV] END .................max_depth=10, min_samples_split=10; total time=   1.5s
[CV] END .................max_depth=10, min_sampl

KeyboardInterrupt: 